In [1]:
import pandas as pd
import numpy as np

import cv2
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from tqdm import tqdm

import torch
from PIL import Image
from torchvision.transforms import v2
from torchvision import transforms

import os

from sklearn import preprocessing


/opt/conda/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

In [2]:
backends = [
  'opencv', 
  'ssd', 
  'dlib', 
  'mtcnn', 
  'retinaface', 
  'mediapipe',
  'yolov8',
  'yunet',
  'fastmtcnn',
]

if torch.cuda.is_available():
    device = 'cuda:0'
else:
    device = 'cpu'

In [3]:
data = pd.read_csv("/kaggle/input/fac-data-p1/labels.csv")
path = "/kaggle/input/fac-data-p1/data/mnt/md0/projects/sami-hackathon/private/data/"
cols = data.columns
y_raw = data[cols[4:]].values
files_list = data['file_name'].values

In [ ]:
data.head()

# label preprocessing

In [4]:
labels_set = {}
for col in cols[4:]:
    temp = data[col].unique()
    labels_set[col] = temp
    
labels_set['age'] = ['Baby', 'Kid', 'Teenager', '20-30s', '40-50s', 'Senior']
print(labels_set)

labels = dict()
for col in cols[4:]:
    enc = preprocessing.OneHotEncoder(categories=[labels_set[col]]).fit(data[col].values.reshape(-1 ,1))
    labels[col] = enc.transform(data[col].values.reshape(-1 ,1)).toarray()
    print(enc.categories_)

temp = data[['file_name', 'bbox']].values
file_names = temp[:, 0]
bboxs = temp[:, 1]
bbxs = []
for i in range(len(bboxs)):
    bbxs.append(eval(bboxs[i]))
bboxes = np.array(bbxs, int)
    

{'age': ['Baby', 'Kid', 'Teenager', '20-30s', '40-50s', 'Senior'], 'race': array(['Caucasian', 'Mongoloid', 'Negroid'], dtype=object), 'masked': array(['unmasked', 'masked'], dtype=object), 'skintone': array(['mid-light', 'light', 'mid-dark', 'dark'], dtype=object), 'emotion': array(['Neutral', 'Happiness', 'Anger', 'Surprise', 'Fear', 'Sadness',
       'Disgust'], dtype=object), 'gender': array(['Male', 'Female'], dtype=object)}
[array(['Baby', 'Kid', 'Teenager', '20-30s', '40-50s', 'Senior'],
      dtype=object)]
[array(['Caucasian', 'Mongoloid', 'Negroid'], dtype=object)]
[array(['unmasked', 'masked'], dtype=object)]
[array(['mid-light', 'light', 'mid-dark', 'dark'], dtype=object)]
[array(['Neutral', 'Happiness', 'Anger', 'Surprise', 'Fear', 'Sadness',
       'Disgust'], dtype=object)]
[array(['Male', 'Female'], dtype=object)]


In [5]:
weights = {}
 
for col in cols[4:]:
    #print(col)
    temp = data[col].value_counts()
    #print(temp[labels_set[col]])
    temp = temp[labels_set[col]].values
    classes = len(temp)
    n = np.sum(temp)
    
    temp = n / (classes * temp)
    
    weights[col] = temp

weights


{'age': array([7.39613527, 2.674703  , 4.76057214, 0.22709742, 1.59280067,
        4.00575615]),
 'race': array([0.71817244, 0.68162593, 7.11761971]),
 'masked': array([ 0.51702013, 15.18849206]),
 'skintone': array([ 1.03782538,  0.3650453 ,  4.79636591, 11.29056047]),
 'emotion': array([ 0.45151587,  0.2372687 ,  6.8562472 ,  7.21829326, 19.18546366,
         5.7556391 , 16.56926407]),
 'gender': array([1.59878864, 0.72752328])}

# Data loader

In [7]:

mytransform2 = transforms.Compose([
            #transforms.RandomHorizontalFlip(),
            #transforms.Resize((224, 224)),
            transforms.ToTensor(),  # mmb
        ]
        )
class Dataset2(torch.utils.data.Dataset):

    def __init__(self, data, labels, root):
        self.labels = labels
        self.data = data
        self.root = root


    def __len__(self):
        return len(self.data)


    def __getitem__(self, index):
        X = self.data[index]
        y = dict()
        y["age"] = labels['age'][index]
        y["race"] = labels['race'][index]
        y["masked"] = labels['masked'][index]
        y["skintone"] = labels['skintone'][index]
        y["emotion"] = labels['emotion'][index]
        y["gender"] = labels['gender'][index]

        X = mytransform2(X)

        return X, y

    
mytransform = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),  # mmb
        ]
        )


In [8]:
from torch.utils.data import random_split

data_preprocessed = np.load('/kaggle/input/face-aligned/np_images.npy')

root = "/kaggle/input/pixta-train-face/images_face"
data_set = Dataset2(data_preprocessed, labels, root)

generator1 = torch.Generator().manual_seed(42)
train_set, test_set = random_split(data_set, [0.8, 0.2], generator = generator1)

# Model

In [9]:
import torch.nn.functional as F

class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)

class InceptionBlock(nn.Module):
    def __init__(
        self, 
        in_channels, 
        out_1x1,
        red_3x3,
        out_3x3,
        red_5x5,
        out_5x5,
        out_pool,
    ):
        super(InceptionBlock, self).__init__()
        self.branch1 = BasicConv2d(in_channels, out_1x1, kernel_size=1)
        self.branch2 = nn.Sequential(
            BasicConv2d(in_channels, red_3x3, kernel_size=1, padding=0),
            BasicConv2d(red_3x3, out_3x3, kernel_size=3, padding=1),
        )
        self.branch3 = nn.Sequential(
            BasicConv2d(in_channels, red_5x5, kernel_size=1),
            BasicConv2d(red_5x5, out_5x5, kernel_size=5, padding=2),
        )
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, padding=1, stride=1),
            BasicConv2d(in_channels, out_pool, kernel_size=1),
        )
    
    def forward(self, x):
        branches = (self.branch1, self.branch2, self.branch3, self.branch4)
        return torch.cat([branch(x) for branch in branches], 1)

    
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
           
        self.fc = nn.Sequential(nn.Conv2d(in_planes, in_planes // 16, 1, bias=False),
                               nn.ReLU(),
                               nn.Conv2d(in_planes // 16, in_planes, 1, bias=False))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

    
class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()

        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)
    



In [10]:
class SkintoneRaceModel(nn.Module):
    def __init__(self):
        super(SkintoneRaceModel, self).__init__()
        self.conv5x5x48_1 = BasicConv2d(3, 48, kernel_size = (5, 5), stride = 2, padding = 2)
        self.conv3x3x96_1 = BasicConv2d(48, 96, kernel_size = (3, 3), stride = 1, padding = 1)
        self.conv3x3x192_1 = BasicConv2d(96, 192, kernel_size = (3, 3), stride = 1, padding = 1)
        self.conv3x3x384 = BasicConv2d(192, 384, kernel_size = (3, 3), stride = 1, padding = 1)
        
        self.conv5x5x48 = BasicConv2d(3, 48, kernel_size = (5, 5), stride = 2, padding = 1)
        self.conv3x3x96 = BasicConv2d(48, 96, kernel_size = (3, 3), stride = 1, padding = 1)
        self.conv3x3x192 = BasicConv2d(96, 192, kernel_size = (3, 3), stride = 1, padding = 1)
        
        self.inception1 = InceptionBlock(192, 128, 128, 256, 128, 256, 128)
        self.conv3x3x512 = BasicConv2d(128 + 256 + 256 + 128, 512, kernel_size = (3, 3), stride = 1, padding = 1)
        
        self.gap = torch.nn.AdaptiveAvgPool2d(1)
        
        self.fc_race = nn.Linear(512, 3)
        self.fc_skintone = nn.Linear(384, 4)
        self.dropout = nn.Dropout(0.3)
        
    def forward(self, x):
        x_skin = self.conv5x5x48_1(x)
        x_skin = nn.functional.max_pool2d(x_skin, (2, 2), 2)
        
        x_skin = self.conv3x3x96_1(x_skin)
        x_skin = nn.functional.max_pool2d(x_skin, (2, 2), 2)
        
        x_skin = self.conv3x3x192_1(x_skin)
        x_skin = nn.functional.max_pool2d(x_skin, (2, 2), 2)
        
        x_skin = self.conv3x3x384(x_skin)
        x_skin = nn.functional.max_pool2d(x_skin, (2, 2), 2)
        
        N, C, W, H = x_skin.shape
        x_skin = self.gap(x_skin).view(N, -1)
        x_skin = self.dropout(x_skin)
        x_skin = self.fc_skintone(x_skin)
        x_skin = nn.functional.softmax(x_skin, dim = 1)
            
        #  RACE
        x_race = self.conv5x5x48(x)
        x_race = nn.functional.max_pool2d(x_race, (2, 2), 2)
        
        x_race = self.conv3x3x96(x_race)
        x_race = nn.functional.max_pool2d(x_race, (2, 2), 2)
        
        x_race = self.conv3x3x192(x_race)
        x_race = nn.functional.max_pool2d(x_race, (2, 2), 2)
        
        x_race = self.inception1(x_race)
        x_race = nn.functional.max_pool2d(x_race, (2, 2), 2)
        # 1152 x 14 x 14
        
        x_race = self.conv3x3x512(x_race)
        x_race = nn.functional.max_pool2d(x_race, (2, 2), 2)
        
        N, C, W, H = x_race.shape
        #print(x.shape)
        #x = x.view(N, -1)
        x_race = self.gap(x_race).view(N, -1)
        x_race = self.dropout(x_race)
        x_race = self.fc_race(x_race)
        x_race = nn.functional.softmax(x_race, dim = 1)
        
        return {"race": x_race, "skintone": x_skin}

In [11]:
class MaskedModel(nn.Module):
    def __init__(self,  att_out = False):
        super(MaskedModel, self).__init__()
        self.att_out = att_out
        self.conv5x5x3 = BasicConv2d(3, 3, kernel_size = (5, 5), stride = 1, padding = 2)
        self.conv3x3x96 = BasicConv2d(3, 96, kernel_size = (3, 3), stride = 1, padding = 1)
        self.conv3x3x192 = BasicConv2d(96, 192, kernel_size = (3, 3), stride = 1, padding = 1)
        self.conv3x3x256 = BasicConv2d(192, 256, kernel_size = (3, 3), stride = 1, padding = 1)
        self.inception1 = InceptionBlock(256, 192, 128, 384, 96, 384, 192)
        
        self.spatial_module = SpatialAttention()
        
        self.gap = torch.nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(192 + 384 + 384 + 192, 2)
        self.dropout = nn.Dropout(0.3)
        
    def forward(self, x):
        #x: 3 x 224 x 224
        x = self.conv5x5x3(x)
        x = nn.functional.max_pool2d(x, (2, 2), 2)
        
        x = self.conv3x3x96(x)
        x = nn.functional.max_pool2d(x, (2, 2), 2)
        
        x = self.conv3x3x192(x)
        x = nn.functional.max_pool2d(x, (2, 2), 2)
        
        x = self.conv3x3x256(x)
        x = nn.functional.max_pool2d(x, (2, 2), 2)
        
        x = self.inception1(x)
        x = nn.functional.max_pool2d(x, (2, 2), 2)
        
        N, C, W, H = x.shape
        sp_att = self.spatial_module(x).view(-1, 1, W, H)

        sp_att2 = sp_att.expand(-1, C, W, H)
        x = x * sp_att2
        
        # 896 x 7 x 7
        x = self.gap(x).view(-1, C)
        x = self.dropout(x)
        
        x = self.fc(x)
        
        x = nn.functional.softmax(x, dim = 1)
        
        if self.att_out:
            return {"masked": x}, sp_att
        
        return {"masked": x}

In [12]:
class GenderModel(nn.Module):
    def __init__(self):
        super(GenderModel, self).__init__()
        self.conv5x5x3 = BasicConv2d(3, 3, kernel_size = (5, 5), stride = 1, padding = 2)
        self.conv3x3x96 = BasicConv2d(3, 96, kernel_size = (3, 3), stride = 1, padding = 1)
        self.conv3x3x192 = BasicConv2d(96, 192, kernel_size = (3, 3), stride = 1, padding = 1)
        self.conv3x3x384 = BasicConv2d(192, 384, kernel_size = (3, 3), stride = 1, padding = 1)
        self.inception1 = InceptionBlock(384, 192, 128, 384, 128, 384, 192)
        
        self.gap = torch.nn.AdaptiveAvgPool2d(1)
        
        self.fc = nn.Linear(192 + 384 + 384 + 192, 2)
        self.dropout = nn.Dropout(0.3)
        
    def forward(self, x):
        #x: 3 x 224 x 224
        x = self.conv5x5x3(x)
        x = nn.functional.max_pool2d(x, (2, 2), 2)
        
        x = self.conv3x3x96(x)
        x = nn.functional.max_pool2d(x, (2, 2), 2)
        
        x = self.conv3x3x192(x)
        x = nn.functional.max_pool2d(x, (2, 2), 2)
        
        x = self.conv3x3x384(x)
        x = nn.functional.max_pool2d(x, (2, 2), 2)
        
        x = self.inception1(x)
        x = nn.functional.max_pool2d(x, (2, 2), 2)
        
        N, C, W, H = x.shape
        # 896 x 7 x 7
        x = self.gap(x).view(N, C)
        x = self.dropout(x)
        
        x = self.fc(x)
        
        x = nn.functional.softmax(x, dim = 1)
           
        return {"gender": x}

In [13]:
masked_model = MaskedModel(False)

In [14]:
skin_race_model = SkintoneRaceModel()

In [15]:
gender_model = GenderModel()

In [16]:
import os
def checkpoint_save(name, state_dict, epoch):
    if not os.path.exists("checkpoint"):
        os.mkdir("checkpoint")
    save_path = "./checkpoint/" + name + "_epoch_{}".format(epoch)
    torch.save(state_dict, save_path)


In [17]:
training_generator = torch.utils.data.DataLoader(train_set, batch_size = 16, shuffle = False)
testing_generator = torch.utils.data.DataLoader(test_set, batch_size = 1, shuffle = False)

# Train function

In [18]:
def convert_result(y_pred):

    results = []
    for i in range(len(y_pred)):
        temp = y_pred[i] > 0.5
        temp = temp.astype(int)
        
        results.append(temp )

    return np.array(results )

In [19]:
import time

def test_model(model):
    model = model.to(device)
    criterion_crossEntropy = torch.nn.CrossEntropyLoss().to(device)
    
    start_time = time.time()

    
    test_loss = 0
    model.eval()
    y_predict = {'age': [], 'gender': [], 'masked': [], 'skintone': [], 'race': [], 'emotion': []}
    y_true = {'age': [], 'gender': [], 'masked': [], 'skintone': [], 'race': [], 'emotion': []}
    
    with torch.no_grad() :
        for b, (X, y) in tqdm(enumerate(testing_generator), total=len(testing_generator)):
            y_pred = model(X.to(device))
            
            loss = 0
            for key in y_pred.keys():
                loss += criterion_crossEntropy(y_pred[key], y[key].to(device))

                test_loss += loss.item()
                
                results = convert_result(y_pred[key].cpu().numpy())
       
                y_predict[key].extend(results)
                y_true[key].extend(y[key].numpy())
            
        print(f'\nDuration: {time.time() - start_time:.0f} seconds')  # print the time elapsed
    return test_loss, y_true, y_predict



In [20]:
def train_model(model, epochs = 10, steps = 256, checkpoint = None, test = True):
    start_time = time.time()
    
    model = model.to(device)
    criterion_crossEntropy = torch.nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    
    running_loss = 0
    train_acc, train_loss = [], [0] * epochs
    val_loss, val_acc = [0] * epochs, []

    for i in range(epochs):

        result_epoch = []
        #train
        model.train()
        for b, (X, y) in tqdm(enumerate(training_generator), total=steps):
            optimizer.zero_grad()

            b += 1
            if b > steps:
                break
            y_pred = model(X.to(device))
            #print(y_pred)
            #print(y)
            loss = 0
            for key in y_pred.keys():
                #print(key)
                #print(y_pred[key], y[key])
                loss += F.cross_entropy(y_pred[key], y[key].to(device), \
                                        weight = torch.Tensor(weights[key]).to(device) ) 
            
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            train_loss[i] += loss.item()

            if b % 50 == 0:

                print(f"Loss: {running_loss / 50} ", )

                running_loss = 0
        train_acc.append(np.sum(result_epoch) / np.array(result_epoch).size)
        if test == True:
            test_loss, y_true, y_predict = test_model(model)
            
            print(f"Test Loss: {test_loss} ", )
            
        if (i + 1) % 5 == 0:
            for param_group in optimizer.param_groups:
                param_group['lr'] = param_group['lr'] * 0.95
            if checkpoint is not None:
                checkpoint_save(checkpoint, model.state_dict(), i)

        #if save_checkpoint:
        #    checkpoint_save(name, model.state_dict(), i)

    print(f'\nDuration: {time.time() - start_time:.0f} seconds')  # print the time elapsed
    
    return train_loss


# Train

In [22]:
losses_gender = train_model(gender_model, epochs = 50, checkpoint = "gender_model", test = False)

 21%|██        | 54/256 [00:01<00:07, 28.09it/s]

Loss: 0.5758844950952136 


 41%|████      | 105/256 [00:03<00:05, 27.62it/s]

Loss: 0.5657463795849018 


 60%|█████▉    | 153/256 [00:05<00:03, 27.68it/s]

...

Loss: 0.3315245981008806 


100%|█████████▉| 255/256 [00:09<00:00, 28.25it/s]

Loss: 0.325472168167094 


100%|██████████| 256/256 [00:09<00:00, 28.14it/s]


Duration: 452 seconds


In [23]:
losses_skin_race = train_model(skin_race_model, epochs = 50, checkpoint = 'skintone_race_model', test = False)

 21%|██        | 53/256 [00:01<00:06, 32.44it/s]

Loss: 2.292836480869581 


 41%|████      | 105/256 [00:03<00:04, 32.61it/s]

Loss: 2.192550647109042 


 60%|█████▉    | 153/256 [00:04<00:03, 32.40it/s]

...

Loss: 1.4594259805145027 


 80%|███████▉  | 204/256 [00:06<00:01, 32.77it/s]

Loss: 1.37765985069257 


100%|██████████| 256/256 [00:07<00:00, 32.38it/s]

Loss: 1.533413795145947 

Duration: 396 seconds


In [24]:
losses_masked = train_model(masked_model, epochs = 50, checkpoint = "masked_model", test = False)

 21%|██▏       | 55/256 [00:02<00:06, 28.77it/s]

Loss: 0.7125947646184057 


 40%|████      | 103/256 [00:03<00:05, 28.82it/s]

Loss: 0.4960687019302555 


 60%|██████    | 154/256 [00:05<00:03, 28.48it/s]

...

Loss: 0.3070672051339471 


100%|█████████▉| 255/256 [00:09<00:00, 28.08it/s]

Loss: 0.32531790337928945 


100%|██████████| 256/256 [00:09<00:00, 28.10it/s]


Duration: 454 seconds


# test

In [39]:
from sklearn.metrics import classification_report, confusion_matrix
def test(model):
    loss, y_true, y_predict = test_model(model)
    print("test loss: ", loss / len(test_set))
    for key in y_predict.keys():
        if len(y_predict[key]) > 1:
            print(key)
            print(classification_report(y_true[key], y_predict[key]))

In [40]:
test(skin_race_model)

100%|██████████| 3062/3062 [00:10<00:00, 283.72it/s]


Duration: 11 seconds
test loss:  2.5538007240720546
skintone
              precision    recall  f1-score   support

           0       0.43      0.72      0.54       730
           1       0.94      0.69      0.79      2095
           2       0.34      0.38      0.36       172
           3       0.44      0.49      0.46        65

   micro avg       0.68      0.67      0.68      3062
   macro avg       0.54      0.57      0.54      3062
weighted avg       0.77      0.67      0.70      3062
 samples avg       0.67      0.67      0.67      3062

race
              precision    recall  f1-score   support

           0       0.94      0.64      0.77      1431
           1       0.77      0.94      0.84      1495
           2       0.45      0.78      0.57       136

   micro avg       0.80      0.80      0.80      3062
   macro avg       0.72      0.79      0.73      3062
weighted avg       0.83      0.80      0.80      3062
 samples avg       0.80      0.80      0.80      3062




/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [41]:
test(gender_model)

100%|██████████| 3062/3062 [00:07<00:00, 410.58it/s]



Duration: 7 seconds
test loss:  0.4208441230399713
gender
              precision    recall  f1-score   support

           0       0.86      0.77      0.81       975
           1       0.90      0.94      0.92      2087

   micro avg       0.89      0.89      0.89      3062
   macro avg       0.88      0.86      0.87      3062
weighted avg       0.89      0.89      0.89      3062
 samples avg       0.89      0.89      0.89      3062



In [42]:
test(masked_model)

100%|██████████| 3062/3062 [00:07<00:00, 393.78it/s]


Duration: 8 seconds
test loss:  0.32239204365820917
masked
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      2957
           1       0.89      0.86      0.87       105

   micro avg       0.99      0.99      0.99      3062
   macro avg       0.94      0.93      0.93      3062
weighted avg       0.99      0.99      0.99      3062
 samples avg       0.99      0.99      0.99      3062



In [ ]:
masked_model.att_out = True

In [ ]:
#mask 3
index = 24
with torch.no_grad():

    y, att = masked_model(train_set[index][0].unsqueeze(0).to(device))
    img = train_set[index][0].permute(1, 2, 0).numpy()
    print(y)
    att = att.cpu().numpy().reshape(7, 7)
    plt.subplot(1, 2, 1)
    plt.imshow(att)
    plt.subplot(1, 2, 2)
    plt.imshow(img)